In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import math
from scipy import stats
from scipy.integrate import quad
from scipy.stats import norm

# Case 6: LIBOR Market Model

## Part 1: Discount Bond Prices

In [2]:
sigma = 0.01  
numSteps = 10  
numSimulations = 1000
dT = 1
f0 = 0.03
N = 10

In [ ]:
# Exercise 1.1
maturities = np.arange(1, N + 1)
discountBond = [1 / (1 + dT*f0)**m for m in maturities]

# Create DataFrame
discountBond_df = pd.DataFrame({
    "T": maturities,
    "Discount Bond Price": discountBond
})
pd.set_option("display.precision", 15)

discountBond_df.head(10)


,T,Discount Bond Price
0,1,0.970873786407767
1,2,0.942595909133754
2,3,0.915141659353160
3,4,0.888487047915689
4,5,0.862608784384164
5,6,0.837484256683654
6,7,0.813091511343354
7,8,0.789409234313936
8,9,0.766416732343627
9,10,0.744093914896725


In [31]:
def discountBond_prices(f0, dT, N):
    return np.array([(1 / (1 + dT*f0)) ** i for i in range(1, N + 1)])

print(discountBond_prices(f0 = 0.03, dT=1, N=10))

[0.97087379 0.94259591 0.91514166 0.88848705 0.86260878 0.83748426
 0.81309151 0.78940923 0.76641673 0.74409391]


In [ ]:
def monte_carlo_discount_bonds(f0, sigma, dt, N, M):
    """   
    Parameters:
        f0 (float): Initial forward rate.
        sigma (float): Volatility.
        N (int): Number of forward rates.
        M (int): Number of Monte Carlo paths.
        steps (int): Number of time steps.
        dt (float): Time step size.
        time_step (int): The time step at which to compute bond prices.

    Returns:
        tuple: Mean and standard error of discount bond prices at the specified time.
    """

    time_step = int(N / dt)

    # Step 1: Simulate forward rates
    f = np.full((M, N), f0)
    f_history = np.zeros((M, N, time_step + 1))
    f_history[:, :, 0] = f


    #MC sim
    drift = np.array([-sigma**2 * (N - i - 1) for i in range(N)])
    # np.random.seed(0)
    Z = np.random.normal(size=(M, time_step))

    for n in range(time_step):
        dW = np.sqrt(dt) * Z[:, n]
        f = f + drift * dt + sigma * dW[:, np.newaxis]
        f_history[:, :, n + 1] = f

    # Step 2: Compute discount bond prices at the specified time step
    f_t = f_history[:, :, time_step]
    D_t_T = np.ones((f_t.shape[0], N))

    for i in range(N):
        D_t_T[:, i] = np.prod(1 / (1 + f_t[:, i:N]), axis=1)

    # Step 3: Summarize the results
    mean = np.mean(D_t_T, axis=0)
    std = np.std(D_t_T, axis=0)
    se = std / np.sqrt(D_t_T.shape[0])

    return mean, se

D_t_T_mean, D_t_T_se = monte_carlo_discount_bonds(f0, sigma, dT, N=10, M=10000)

print("\nDiscount bond prices at t = dt:")
for i in range(N):
    print(f"D(dt, T_{i+1}) Mean = {D_t_T_mean[i]:}, SE = {D_t_T_se[i]:}")


IndexError: index 10 is out of bounds for axis 1 with size 10

In [ ]:
# Initialize matrices
discountFactors = np.ones((N + 1, numSteps, numSimulations))  
dW = np.random.normal(size=(numSteps - 1, numSimulations))  # Random Brownian increments

# Initial LIBOR rates
LIBOR = np.zeros((N, numSteps, numSimulations)) 
LIBOR[:, 0, :] = f0  

# Simulate LIBOR rates
for t in range(1, numSteps):
    for i in range(N):
        # Drift term
        drift = -sum(
            (dT * sigma) / (1 + dT * LIBOR[j, t - 1, :])
            for j in range(i + 1, N)
        )
        # Update LIBOR rates
        LIBOR[i, t, :] = LIBOR[i, t - 1, :] + drift * sigma * dT + sigma * np.sqrt(dT) * dW[t - 1, :]

# Compute discount factors
for t in range(1, numSteps):
    for i in range(1, N + 1):
        discountFactors[i, t, :] = discountFactors[i - 1, t, :] / (1 + dT * LIBOR[i - 1, t, :])

print("Monte-Carlo simulated LIBOR rates:", LIBOR)
print("Discount factors:", discountFactors)

Monte-Carlo simulated LIBOR rates: [[[ 0.03        0.03        0.03       ...  0.03        0.03
    0.03      ]
  [ 0.0239961   0.02561582  0.03220898 ...  0.00507081  0.02948748
    0.0552376 ]
  [ 0.03271272  0.02236055  0.02284069 ... -0.00630246  0.0445876
    0.04874479]
  ...
  [ 0.03456013 -0.01709867  0.01073882 ...  0.01645011  0.04031552
    0.08054146]
  [ 0.03726147 -0.00195072  0.00089716 ...  0.01268913  0.03363641
    0.08824465]
  [ 0.01929822 -0.01107412  0.01333433 ...  0.00565176  0.02989638
    0.08299771]]

 [[ 0.03        0.03        0.03       ...  0.03        0.03
    0.03      ]
  [ 0.02409318  0.0257129   0.03230606 ...  0.00516789  0.02958457
    0.05533469]
  [ 0.03290745  0.02255513  0.02303465 ... -0.00610589  0.04478181
    0.04893663]
  ...
  [ 0.03523606 -0.01640303  0.01142276 ...  0.01714945  0.040986
    0.0812074 ]
  [ 0.03803399 -0.00115341  0.00167998 ...  0.01348679  0.03440295
    0.08900309]
  [ 0.02016708 -0.0101767   0.01421698 ...  0.0065480

## Part 2: Gaussian swaption formulas

In [10]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# Define the Gaussian swaption price function
def gaussian_swaption_price(T0, TN, K, sigma, f0, L=1.0):
    """
    Calculate payer swaption price using Gaussian formula
    
    Parameters:
    T0 - Option maturity
    TN - Swap maturity
    K - Strike rate
    sigma - Volatility
    f0 - Initial forward rate
    L - Notional (default 1.0)
    """
    # Calculate discount factors
    D = np.zeros(TN + 1)
    D[0] = 1.0
    for i in range(1, TN + 1):
        D[i] = D[i - 1] / (1 + f0)
    
    # Calculate sum of discount factors for swap
    P_sum = np.sum(D[T0 + 1:TN + 1])
    
    # Calculate forward swap rate
    y_T0TN = (D[T0] - D[TN]) / P_sum
    
    # Calculate d parameter
    d = (y_T0TN - K) / (sigma * np.sqrt(T0))
    
    # Normal density function
    phi = np.exp(-0.5 * d**2) / np.sqrt(2 * np.pi)
    
    # Calculate swaption price
    price = L * P_sum * ((y_T0TN - K) * norm.cdf(d) + sigma * np.sqrt(T0) * phi)
    
    return price

# Parameters for calculation
maturities = range(1, 10)  # T1 to T9
strikes = np.arange(0.01, 0.06, 0.01)  # 1% to 5%
sigma = 0.01
f0 = 0.03

# Calculate swaption prices
results = np.zeros((len(maturities), len(strikes)))

for i, T in enumerate(maturities):
    for j, K in enumerate(strikes):
        results[i, j] = gaussian_swaption_price(T, 10, K, sigma, f0)

# Convert results into a DataFrame
df_results = pd.DataFrame(
    results,
    index=[f"T{i+1}" for i in range(len(maturities))],
    columns=[f"{int(K*100)}%" for K in strikes]
)

df_results.head(11)

,1%,2%,3%,4%,5%
T1,0.151828421156856,0.081891381075267,0.030157359696586,0.006298090571586,0.000641840149494
T2,0.135659871737680,0.079377058733749,0.037330919153996,0.013209727321406,0.003325208912993
T3,0.120106997531761,0.074295017496134,0.039397329993311,0.017279102677322,0.006075167894138
T4,0.104282209897379,0.067171354088681,0.038403017173929,0.019040309749027,0.008020121218070
T5,0.087972549047613,0.058459857576332,0.035240872795873,0.018954901080519,0.008962636055987
T6,0.071162874439025,0.048485971942933,0.030420503723044,0.017355858013956,0.008902646581072
T7,0.053898485210447,0.037488928436350,0.024275701760611,0.014489729620807,0.007900087579362
T8,0.036241416966556,0.025651149294993,0.017044287460220,0.010546042822590,0.006031204021749
T9,0.018255294594077,0.013116205516875,0.008905515697252,0.005675266367908,0.003373416296142
